In [120]:
#import Dependencies
import pandas as pd
import os


In [121]:
#create a varialbe for the file
file = "purchase_data.csv"

In [122]:
#import and view the first 5 lines of the data file
purchase_data_df = pd.read_csv(file)

In [123]:
#get player count
#break to a new df with only the data we need
player_count_df = purchase_data_df.copy()["SN"]
#count unique Screen Names
player_count = player_count_df.nunique()


In [124]:
#Purchasing Analysis(Total)
#Number of Unique Items
number_items_df = purchase_data_df.copy()["Item Name"]
number_items = number_items_df.nunique()

#Average Purchase Price
average_purcahse_df = purchase_data_df.copy()["Price"]
average_purchase = average_purcahse_df.mean()

#Total Number of Purchases

num_purchases_df = purchase_data_df.copy()["Purchase ID"]
num_purchases = num_purchases_df.count()

#Total Revenue
rev_df = purchase_data_df.copy()["Price"]
rev = rev_df.sum()

average_purchase = "${:,.2f}".format(average_purchase)
rev = "${:,.2f}".format(rev)
purchasing_summary = pd.DataFrame({
    "Number of Unique Itesm":[number_items], "Average Price": [average_purchase], "Number of Purchases": num_purchases, "Total Revenus": [rev]
})
purchasing_summary

,Number of Unique Itesm,Average Price,Number of Purchases,Total Revenus
0,179,$3.05,780,"$2,379.77"


In [125]:
#Gender Demographics

grouped_on_SN = purchase_data_df.copy().drop_duplicates(subset=['SN'])

#Percentage and Count of Male Players
male_count = grouped_on_SN.loc[purchase_data_df.copy()["Gender"] == 'Male',:].count()
male_count = male_count["Gender"]
male_percent = male_count/player_count
male_percent = "{:.2%}".format(male_percent)

#Percentage and Count of Female Players
female_count = grouped_on_SN.loc[purchase_data_df.copy()["Gender"] == 'Female',:].count()
female_count = female_count["Gender"]
female_percent = female_count/player_count
female_percent = "{:.2%}".format(female_percent)

#Percentage and Count of Other / Non-Disclosed
other_count = grouped_on_SN.loc[purchase_data_df.copy()["Gender"] == 'Other / Non-Disclosed',:].count()
other_count = other_count["Gender"]
other_percent = other_count/player_count
other_percent = "{:.2%}".format(other_percent)

gender_demo_summary = pd.DataFrame({
    "Gender":["Male", "Female", "Other / Non-Disclosed"],
    "Total Count": [male_count, female_count, other_count],
    "Percentage of Players": [male_percent, female_percent, other_percent]
})

gender_demo_summary

,Gender,Total Count,Percentage of Players
0,Male,484,84.03%
1,Female,81,14.06%
2,Other / Non-Disclosed,11,1.91%


In [134]:
#Purchasing Analysis (Gender)
#The below each broken by gender
gender_grouped_df = purchase_data_df.copy().groupby(["Gender"])
#Purchase Count
purchase_by_gender = gender_grouped_df["Purchase ID"].count()

#Average Purchase Price
avg_by_gender = gender_grouped_df["Price"].mean()

#Total Purchase Value
total_by_gender = gender_grouped_df["Price"].sum()

#Average Purchase Total per Person by Gender






#Summary DF + Formating
gender_purchase_summary = pd.DataFrame({
    "Number of Purchases": purchase_by_gender,
    "Average Purchase": avg_by_gender, 
    "Total Purchase": total_by_gender
})
gender_purchase_summary = gender_purchase_summary.merge(gender_demo_summary,how='left', on="Gender")
gender_purchase_summary["Avg Total Purchase Per Person"] = gender_purchase_summary["Total Purchase"]/gender_purchase_summary["Total Count"]

gender_purchase_summary = gender_purchase_summary[["Gender", "Number of Purchases", "Average Purchase", "Total Purchase","Avg Total Purchase Per Person" ]]
gender_purchase_summary["Average Purchase"] = gender_purchase_summary["Average Purchase"].map("${:.2f}".format)
gender_purchase_summary["Total Purchase"] = gender_purchase_summary["Total Purchase"].map("${:.2f}".format)
gender_purchase_summary["Avg Total Purchase Per Person"] = gender_purchase_summary["Avg Total Purchase Per Person"].map("${:.2f}".format)
gender_purchase_summary

,Gender,Number of Purchases,Average Purchase,Total Purchase,Avg Total Purchase Per Person
0,Female,113,$3.20,$361.94,$4.47
1,Male,652,$3.02,$1967.64,$4.07
2,Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [175]:
#Age Demographics
#The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)
age_bins = [0,9, 14,19, 24, 29,34, 39, 100]
age_labels = ["Under 10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "Over 40"]

age_df = purchase_data_df.copy().drop_duplicates(subset=['SN'])
age_df["age_groups"] = pd.cut(age_df["Age"], age_bins, labels=age_labels)

#Purchase Count
age_count = age_df.groupby("age_groups").count()

age_count = age_count.rename(columns={"Age":"Total Count"})
age_count
#Percentage of Players
age_count["Number of Players"] = player_count
age_count["Percentage of Players"] = (age_count["Total Count"]/age_count["Number of Players"])*100
age_count["Percentage of Players"] = age_count["Percentage of Players"].map("{:.2f}%".format)
age_count = age_count[['Total Count',"Percentage of Players"]]
age_count

,Total Count,Percentage of Players
age_groups,,
Under 10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
Over 40,12,2.08%


In [ ]:
#Average Purchase Price

#Total Purchase Value

#Average Purchase Total per Person by Age Group
